In [31]:
model *LVModel()
  // Reactions:
  R1: Magikarp -> 2Magikarp; kprey*Magikarp;
  R2: Magikarp -> Pidgeot; kpred*Magikarp*Pidgeot;
  R4: Pidgeot -> ; kd*Pidgeot;

  // Species initializations:
  Magikarp = 71;
  Pidgeot = 79;

  // Variable initializations:
  kprey = 0.5;
  kpred = 0.0025;
  kd = 0.3;
end

In [32]:
LVModel.reset()
sim = LVModel.simulate(0,50,50)
LVModel.plot()

In [33]:
import numpy as np
nrows = sim.shape[0]
print('nrows = {}'.format(nrows))
noise = np.random.normal(0., 100., (nrows,1))

In [34]:
import matplotlib.pyplot as plt
plt.plot(noise)

In [35]:
noisy_M = np.reshape(sim[:,1],(nrows,1)) + noise
noisy_P = np.reshape(sim[:,2],(nrows,1)) + noise

In [36]:
plt.plot(sim[:,0], noisy_M)
plt.plot(sim[:,0], noisy_P)


In [37]:
%cd ~
import csv
with open('LVData.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # write header row
    writer.writerow(['time', 'M (noisy)', 'B (noisy)'])
    for row in range(len(noisy_M)):
        writer.writerow([sim[row,0], float(noisy_M[row]), float(noisy_P[row])])

In [38]:
times = np.zeros(50)
M_data = np.zeros(50)
P_data = np.zeros(50)

with open('LVData.csv') as csvfile:
    reader = csv.reader(csvfile)
    k = 0
    for row in reader:
        break
    for row in reader:
        times[k] = row[0]
        M_data[k] = row[1]
        P_data[k] = row[2]
        k += 1

In [39]:
model *LVModel()
  // Reactions:
  R1: Magikarp -> 2Magikarp; kprey*Magikarp;
  R2: Magikarp -> Pidgeot; kpred*Magikarp*Pidgeot;
  R4: Pidgeot -> ; kd*Pidgeot;

  // Species initializations:
  Magikarp = 71;
  Pidgeot = 79;

  // Variable initializations:
  kprey = 1;
  kpred = 0.01;
  kd = 1;
end

In [40]:
LVModel.reset()
LVModel.simulate(0,50,50)
LVModel.plot()

In [41]:
from scipy.optimize import differential_evolution

In [42]:
def mse(params):
    LVModel.kprey = params[0]
    LVModel.kpred = params[1]
    LVModel.kd = params[2]
    LVModel.reset()
    sim = LVModel.simulate(0,50,50)
    A_mse = np.sqrt(np.mean(np.square(sim[:,1] - M_data)))
    B_mse = np.sqrt(np.mean(np.square(sim[:,2] - P_data)))
    mse = np.sqrt(np.square(A_mse) + np.square(B_mse))
    return float(mse)

In [43]:
bounds = [
    (0.3, 1.5), # kprey true value = 0.5
    (0.0025, 0.0125), # kpred true value = 0.0025
    (0.3, 1.0) # kd true value = 0.3
]

In [44]:
ret = differential_evolution(mse, bounds)


In [45]:
ret.x